In [1]:
#%load_ext watermark
%load_ext autoreload 
%autoreload 2

In [2]:
import os
import sys
import itertools
import numpy as np
import pandas as pd
from subprocess import call
# import pybpr
import matplotlib.pyplot as plt
from functools import partial
from sklearn.feature_extraction import DictVectorizer
from scipy.special import expit

In [3]:
item_df = pd.DataFrame.from_dict(dict(
    item1=[1,0,0,1],
    item2=[1,0,0,1],
    item3=[0,1,1,0],
    item4=[0,1,1,0],
    item5=[0,1,1,0]
), orient='index', columns=[f'feature{i+1}' for i in range(4)])
n_items = item_df.shape[0]
n_features = item_df.shape[1]
print(n_items, n_features)
item_df

5 4


,feature1,feature2,feature3,feature4
item1,1,0,0,1
item2,1,0,0,1
item3,0,1,1,0
item4,0,1,1,0
item5,0,1,1,0


In [4]:
item_df_identity = pd.DataFrame(np.eye(item_df.shape[0]), dtype=int, index=item_df.index, columns=item_df.index)
item_df = pd.concat([item_df_identity, item_df], axis=1)
item_df

,item1,item2,item3,item4,item5,feature1,feature2,feature3,feature4
item1,1,0,0,0,0,1,0,0,1
item2,0,1,0,0,0,1,0,0,1
item3,0,0,1,0,0,0,1,1,0
item4,0,0,0,1,0,0,1,1,0
item5,0,0,0,0,1,0,1,1,0


In [5]:
user_df = pd.DataFrame(np.eye(3), dtype=int, index=[f'user{i+1}' for i in range(3)])
user_df.columns = list(user_df.index.values)
n_users = user_df.shape[0]
user_df

,user1,user2,user3
user1,1,0,0
user2,0,1,0
user3,0,0,1


In [6]:
# int_df = pd.DataFrame.from_dict(dict(
#     user1=[1,0,0,0,0],
#     user2=[0,0,1,0,0],
#     #user3=[1,0,1,1,0],
# ), orient='index', columns=[f'item{i+1}' for i in range(5)])
# int_df

In [7]:
#initiate
nlatent = 2
user_latent_mat = np.random.normal(loc=0.,scale=0.1,size=(user_df.shape[1],nlatent))
item_latent_mat = np.random.normal(loc=0.,scale=0.1,size=(item_df.shape[1],nlatent))
print(item_latent_mat.shape, user_latent_mat.shape)
np.round(item_latent_mat,2)

(9, 2) (3, 2)


array([[-0.03,  0.06],
       [ 0.06, -0.09],
       [ 0.17,  0.12],
       [-0.11, -0.03],
       [ 0.15,  0.03],
       [-0.18, -0.14],
       [-0.01,  0.06],
       [-0.1 , -0.03],
       [-0.01,  0.05]])

In [8]:
item_df.values[:, item_df.shape[0]:]

array([[1, 0, 0, 1],
       [1, 0, 0, 1],
       [0, 1, 1, 0],
       [0, 1, 1, 0],
       [0, 1, 1, 0]])

In [9]:
from sklearn.decomposition import PCA
pca = PCA(n_components=nlatent).fit(item_df.values[:, item_df.shape[0]:])
pca_vecs = np.round(pca.components_,3)
pca_vecs.T

array([[ 0.5  , -0.866],
       [-0.5  , -0.289],
       [-0.5  , -0.289],
       [ 0.5  ,  0.289]])

In [10]:
np.round(np.dot(pca_vecs[0,:], pca_vecs[1,:]),2)

0.0

In [11]:
item_latent_mat[item_df.shape[0]:,:] = pca.components_.T
np.round(item_latent_mat,2)

array([[-0.03,  0.06],
       [ 0.06, -0.09],
       [ 0.17,  0.12],
       [-0.11, -0.03],
       [ 0.15,  0.03],
       [ 0.5 , -0.87],
       [-0.5 , -0.29],
       [-0.5 , -0.29],
       [ 0.5 ,  0.29]])

In [12]:
np.round(np.linalg.norm(item_latent_mat, axis=1),2)

array([0.06, 0.1 , 0.21, 0.11, 0.15, 1.  , 0.58, 0.58, 0.58])

In [13]:
for ix in range(n_items):
    item_latent_mat[ix,:] /= 100*np.linalg.norm(item_latent_mat[ix,:])
for ix in range(n_features):
    item_latent_mat[n_items+ix,:] /= np.linalg.norm(item_latent_mat[n_items+ix,:])
for ix in range(user_latent_mat.shape[0]):
    user_latent_mat[ix,:] /= 10*np.linalg.norm(user_latent_mat[ix,:])
np.round(np.linalg.norm(item_latent_mat, axis=1),2)
#np.round(np.linalg.norm(user_latent_mat, axis=1),2)

array([0.01, 0.01, 0.01, 0.01, 0.01, 1.  , 1.  , 1.  , 1.  ])

In [14]:
np.round(item_latent_mat,2)

array([[-0.  ,  0.01],
       [ 0.01, -0.01],
       [ 0.01,  0.01],
       [-0.01, -0.  ],
       [ 0.01,  0.  ],
       [ 0.5 , -0.87],
       [-0.87, -0.5 ],
       [-0.87, -0.5 ],
       [ 0.87,  0.5 ]])

In [15]:
np.round(user_latent_mat,2)

array([[ 0.08,  0.06],
       [ 0.08, -0.07],
       [-0.07, -0.08]])

In [16]:
for user_u in range(n_users):
    print('Recs for user: ', user_u)
    rec = user_latent_mat[user_u].dot(item_latent_mat.T)
    for i in range(n_items):
        inds = item_df.iloc[i].values.nonzero()[0]
        print(i, ':', np.round(np.sum(rec[inds]),3))

Recs for user:  0
0 : 0.091
1 : 0.091
2 : -0.198
3 : -0.2
4 : -0.198
Recs for user:  1
0 : 0.126
1 : 0.128
2 : -0.065
3 : -0.066
4 : -0.064
Recs for user:  2
0 : -0.062
1 : -0.062
2 : 0.188
3 : 0.19
4 : 0.188


In [19]:
# interaction
user_u = 0
item_pos = 0
item_neg = 1

In [20]:
#gradients
for ix in range(10): 
    print(f'::::::::Iter{ix}')
    # slice user-item latent mat
    user_latent = user_latent_mat[user_u]
    #print(np.round(user_latent,2))
    pos_inds = item_df.iloc[item_pos].values.nonzero()[0]
    item_latent_pos = item_latent_mat[pos_inds,:]
    #print(np.round(item_latent_pos,2))
    item_latent_pos = np.sum(item_latent_pos,axis=0)
    #print(np.round(item_latent_pos,2))
    neg_inds = item_df.iloc[item_neg].values.nonzero()[0]
    item_latent_neg = item_latent_mat[neg_inds,:]
    #print(np.round(item_latent_neg,2))
    item_latent_neg = np.sum(item_latent_neg, axis=0)
    #print(np.round(item_latent_neg,2))

    r_uij = np.dot(user_latent, (item_latent_pos - item_latent_neg))
    sigmoid = expit(-r_uij)
    sigmoid_tiled = np.tile(sigmoid, (nlatent,))
    grad_u = np.multiply(sigmoid_tiled, (item_latent_neg - item_latent_pos))
    grad_pos = np.multiply(sigmoid_tiled, -user_latent)
    grad_neg = np.multiply(sigmoid_tiled, user_latent)

    # bpr update
    learning_rate=0.1
    user_latent_mat[user_u] -= learning_rate * grad_u
    item_latent_mat[item_pos] -= learning_rate * grad_pos
    item_latent_mat[item_neg] -= learning_rate * grad_neg
    print('User:\n', np.round(user_latent_mat,3))
    print('Item:\n', np.round(item_latent_mat,3))
    rec = user_latent_mat[user_u].dot(item_latent_mat.T)
    for i in range(n_items):
        inds = item_df.iloc[i].values.nonzero()[0]
        print(i, ':', np.round(np.sum(rec[inds]),3))
    #print('Recs:', np.round(rec,3))

::::::::Iter0
User:
 [[ 0.716  0.038]
 [ 0.097 -0.024]
 [-0.028  0.096]]
Item:
 [[ 0.103 -0.017]
 [-0.025 -0.003]
 [-0.075  0.017]
 [ 0.006  0.008]
 [-0.006 -0.008]
 [ 0.5   -0.866]
 [-0.866 -0.5  ]
 [-0.866 -0.5  ]
 [ 0.866  0.5  ]]
0 : 1.037
1 : 0.946
2 : -1.331
3 : -1.273
4 : -1.283
::::::::Iter1
User:
 [[ 0.722  0.038]
 [ 0.097 -0.024]
 [-0.028  0.096]]
Item:
 [[ 0.137 -0.015]
 [-0.059 -0.005]
 [-0.075  0.017]
 [ 0.006  0.008]
 [-0.006 -0.008]
 [ 0.5   -0.866]
 [-0.866 -0.5  ]
 [-0.866 -0.5  ]
 [ 0.866  0.5  ]]
0 : 1.071
1 : 0.93
2 : -1.342
3 : -1.283
4 : -1.292
::::::::Iter2
User:
 [[ 0.731  0.037]
 [ 0.097 -0.024]
 [-0.028  0.096]]
Item:
 [[ 0.171 -0.013]
 [-0.093 -0.007]
 [-0.075  0.017]
 [ 0.006  0.008]
 [-0.006 -0.008]
 [ 0.5   -0.866]
 [-0.866 -0.5  ]
 [-0.866 -0.5  ]
 [ 0.866  0.5  ]]
0 : 1.109
1 : 0.917
2 : -1.358
3 : -1.299
4 : -1.308
::::::::Iter3
User:
 [[ 0.743  0.037]
 [ 0.097 -0.024]
 [-0.028  0.096]]
Item:
 [[ 0.204 -0.012]
 [-0.126 -0.008]
 [-0.075  0.017]
 [ 0.006 